In [ ]:
# default_exp connectomes

Module connectomes
---

Pipeline making use of `postprocessing` nodes to generate a connectivity matrix and distance matrix.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
from nipype import IdentityInterface
from nipype.pipeline import Node, Workflow
from nipype.interfaces.io import SelectFiles, DataSink

201001-10:23:12,863 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [ ]:
# export

class connectome:
    """
    Create a workflow that produces connectomes based on input atlases and streamlines
    
    Inputs:
    atlas_dir (str): base directory of folder containing atlases
    atlas_list (List of strings): names of atlases: aal, brainnectome, desikan-killiany
    
    """
    
    def __init__(self, atlas_dir, atlas_list):
        """
        Initialize workflow nodes
        """
        self.atlas_dir = atlas_dir
        self.atlas_list = atlas_list
        
    def create_nodes(self):
        self.PostProcNodes = nodes.PostProcNodes()
        
    def connect_nodes(self, wf_name="connectomes"):
        """
        Connect post processing nodes and create workflow
        """
        self.workflow = Workflow(name=wf_name, base_dir)
        self.workflow.connect(
            [
                (self.PostProcNodes.subject_source, self.PostProcNodes.select_files, [('subject_id', 'subject_id'),
                                                                                      ('session_id', 'session_id')]),
                (self.PostProcNodes.atlas_source, self.PostProcNodes.select_atlases, [('atlas_name', 'atlas_name')]),
                (),
            ])
        
    def draw_pipeline(self):
        """
        Visualize workflow
        """
        self.workflow.write_graph(graph2use='orig', dotfilename='postprocess.dot')
    
    def run_pipeline(self, parallel=None):
        """
        Run nipype workflow
        """
        if type(parallel) == int:
            print("Running workflow with {} parallel processes".format(parallel))
            self.workflow.run('MultiProc', plugin_args = {'n_procs': parallel})
        elif parallel is None:
            print("Parallel processing disabled, running workflow serially")
            self.workflow.run()